In [753]:
import numpy as np
import pandas as pd 

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

In [754]:
df= pd.read_csv(r"D:\TEK UP 2eme\Machine Learning\kidney_disease.csv")

In [755]:
df.head(4)

,id,age,bp,sg,al,su,rbc,pc,pcc,ba,...,pcv,wc,rc,htn,dm,cad,appet,pe,ane,classification
0,0,48.0,80.0,1.020,1.0,0.0,NaN,normal,notpresent,notpresent,...,44,7800,5.2,yes,yes,no,good,no,no,ckd
1,1,7.0,50.0,1.020,4.0,0.0,NaN,normal,notpresent,notpresent,...,38,6000,NaN,no,no,no,good,no,no,ckd
2,2,62.0,80.0,1.010,2.0,3.0,normal,normal,notpresent,notpresent,...,31,7500,NaN,no,yes,no,poor,no,yes,ckd
3,3,48.0,70.0,1.005,4.0,0.0,normal,abnormal,present,notpresent,...,32,6700,3.9,yes,no,no,poor,yes,yes,ckd


In [756]:
df = df.sample(frac = 1).reset_index(drop=True)

In [757]:
df.shape

(400, 26)

In [758]:
df.isna().sum()

id                  0
age                 9
bp                 12
sg                 47
al                 46
su                 49
rbc               152
pc                 65
pcc                 4
ba                  4
bgr                44
bu                 19
sc                 17
sod                87
pot                88
hemo               52
pcv                70
wc                105
rc                130
htn                 2
dm                  2
cad                 2
appet               1
pe                  1
ane                 1
classification      0
dtype: int64

In [759]:
df.describe()

,id,age,bp,sg,al,su,bgr,bu,sc,sod,pot,hemo
count,400.000000,391.000000,388.000000,353.000000,354.000000,351.000000,356.000000,381.000000,383.000000,313.000000,312.000000,348.000000
mean,199.500000,51.483376,76.469072,1.017408,1.016949,0.450142,148.036517,57.425722,3.072454,137.528754,4.627244,12.526437
std,115.614301,17.169714,13.683637,0.005717,1.352679,1.099191,79.281714,50.503006,5.741126,10.408752,3.193904,2.912587
min,0.000000,2.000000,50.000000,1.005000,0.000000,0.000000,22.000000,1.500000,0.400000,4.500000,2.500000,3.100000
25%,99.750000,42.000000,70.000000,1.010000,0.000000,0.000000,99.000000,27.000000,0.900000,135.000000,3.800000,10.300000
50%,199.500000,55.000000,80.000000,1.020000,0.000000,0.000000,121.000000,42.000000,1.300000,138.000000,4.400000,12.650000
75%,299.250000,64.500000,80.000000,1.020000,2.000000,0.000000,163.000000,66.000000,2.800000,142.000000,4.900000,15.000000
max,399.000000,90.000000,180.000000,1.025000,5.000000,5.000000,490.000000,391.000000,76.000000,163.000000,47.000000,17.800000


In [760]:
df.dtypes

id                  int64
age               float64
bp                float64
sg                float64
al                float64
su                float64
rbc                object
pc                 object
pcc                object
ba                 object
bgr               float64
bu                float64
sc                float64
sod               float64
pot               float64
hemo              float64
pcv                object
wc                 object
rc                 object
htn                object
dm                 object
cad                object
appet              object
pe                 object
ane                object
classification     object
dtype: object

In [761]:
cat_col = []
num_col=[]
for x in df.dtypes.index:
    if df.dtypes[x] == 'object':
        cat_col.append(x)
    else:
        num_col.append(x)
cat_col

['rbc',
 'pc',
 'pcc',
 'ba',
 'pcv',
 'wc',
 'rc',
 'htn',
 'dm',
 'cad',
 'appet',
 'pe',
 'ane',
 'classification']

In [762]:
num_col

['id', 'age', 'bp', 'sg', 'al', 'su', 'bgr', 'bu', 'sc', 'sod', 'pot', 'hemo']

In [763]:
def unique_values_cat(data):
    cat_cols=[c for c in data.dtypes.index if data.dtypes[c] =='object']
    cat_unique={c: data[c].unique() for c in cat_cols}
    return cat_unique

In [764]:
unique_values_cat(df)

{'rbc': array(['normal', nan, 'abnormal'], dtype=object),
 'pc': array(['normal', nan, 'abnormal'], dtype=object),
 'pcc': array(['notpresent', nan, 'present'], dtype=object),
 'ba': array(['notpresent', nan, 'present'], dtype=object),
 'pcv': array(['49', '44', '41', '48', '50', '45', nan, '39', '52', '35', '17',
        '54', '24', '36', '37', '25', '42', '40', '19', '32', '51', '47',
        '29', '53', '27', '43', '34', '22', '28', '46', '23', '30', '31',
        '\t43', '33', '26', '38', '18', '\t?', '16', '14', '20', '9', '21',
        '15'], dtype=object),
 'wc': array(['5100', '7800', '7000', '5000', '7700', nan, '4300', '15200',
        '6500', '9500', '10700', '5400', '9400', '4200', '8600', '9800',
        '7900', '6200', '2200', '10300', '6700', '4500', '6900', '9200',
        '7500', '6300', '6800', '8100', '6400', '5800', '16300', '8500',
        '7200', '4700', '5500', '11000', '10200', '9100', '5600', '8400',
        '19100', '9300', '7100', '15700', '4900', '6600', '10

In [765]:
df=df.replace('\t',"",regex=True)

In [766]:
df=df.replace('?',np.nan)

In [767]:
unique_values_cat(df)

{'rbc': array(['normal', nan, 'abnormal'], dtype=object),
 'pc': array(['normal', nan, 'abnormal'], dtype=object),
 'pcc': array(['notpresent', nan, 'present'], dtype=object),
 'ba': array(['notpresent', nan, 'present'], dtype=object),
 'pcv': array(['49', '44', '41', '48', '50', '45', nan, '39', '52', '35', '17',
        '54', '24', '36', '37', '25', '42', '40', '19', '32', '51', '47',
        '29', '53', '27', '43', '34', '22', '28', '46', '23', '30', '31',
        '33', '26', '38', '18', '16', '14', '20', '9', '21', '15'],
       dtype=object),
 'wc': array(['5100', '7800', '7000', '5000', '7700', nan, '4300', '15200',
        '6500', '9500', '10700', '5400', '9400', '4200', '8600', '9800',
        '7900', '6200', '2200', '10300', '6700', '4500', '6900', '9200',
        '7500', '6300', '6800', '8100', '6400', '5800', '16300', '8500',
        '7200', '4700', '5500', '11000', '10200', '9100', '5600', '8400',
        '19100', '9300', '7100', '15700', '4900', '6600', '10500', '8000',
  

In [768]:
df[['pcv','wc','rc']].isna().any(axis=1).sum()

136

In [769]:
df.wc.isna().sum()

106

lors du conversion des colonne de type object to numerics (celles qu'elles doivent etre numeriques) on a un prob : un erreur raised à cause des 'NAN'.
jai pensé d'effacer les NAN pour ces 3 colonnes, mais on ne peux pas car le nombre est important, donc on essai de faire une autre logique.

In [770]:
def convert_cat_to_num(cols,data):

    for c in cols:
        
        data[c]=data[c].apply(lambda x: float(x) if x != np.nan else x)
       
       

In [771]:
convert_cat_to_num(['pcv','wc','rc'] ,df)

In [772]:
cat_col = []
num_col=[]
for x in df.dtypes.index:
    if df.dtypes[x] == 'object':
        cat_col.append(x)
    else:
        num_col.append(x)
cat_col

['rbc',
 'pc',
 'pcc',
 'ba',
 'htn',
 'dm',
 'cad',
 'appet',
 'pe',
 'ane',
 'classification']

In [773]:
df.isna().sum()

id                  0
age                 9
bp                 12
sg                 47
al                 46
su                 49
rbc               152
pc                 65
pcc                 4
ba                  4
bgr                44
bu                 19
sc                 17
sod                87
pot                88
hemo               52
pcv                71
wc                106
rc                131
htn                 2
dm                  2
cad                 2
appet               1
pe                  1
ane                 1
classification      0
dtype: int64

In [774]:
def impute_missing_values_num(data, columns):
    for column in columns:
        data[column] = data[column].fillna(data[column].mean())

In [775]:
impute_missing_values_num(df, num_col)

In [776]:
unique_values_cat(df)

{'rbc': array(['normal', nan, 'abnormal'], dtype=object),
 'pc': array(['normal', nan, 'abnormal'], dtype=object),
 'pcc': array(['notpresent', nan, 'present'], dtype=object),
 'ba': array(['notpresent', nan, 'present'], dtype=object),
 'htn': array(['no', 'yes', nan], dtype=object),
 'dm': array(['no', 'yes', ' yes', nan], dtype=object),
 'cad': array(['no', 'yes', nan], dtype=object),
 'appet': array(['good', 'poor', nan], dtype=object),
 'pe': array(['no', 'yes', nan], dtype=object),
 'ane': array(['no', 'yes', nan], dtype=object),
 'classification': array(['notckd', 'ckd'], dtype=object)}

In [777]:
df['dm']=df['dm'].apply(lambda x: 'yes' if (x == ' yes') else x)

In [778]:
df['dm'].unique()

array(['no', 'yes', nan], dtype=object)

In [779]:
def impute_missing_values_cat(data, columns):
    for column in columns:
        data[column] = data[column].fillna(data[column].mode()[0])

impute_missing_values_cat(df, cat_col)

In [780]:
df.isna().sum()

id                0
age               0
bp                0
sg                0
al                0
su                0
rbc               0
pc                0
pcc               0
ba                0
bgr               0
bu                0
sc                0
sod               0
pot               0
hemo              0
pcv               0
wc                0
rc                0
htn               0
dm                0
cad               0
appet             0
pe                0
ane               0
classification    0
dtype: int64

In [781]:
y=df["classification"]
x=df.drop("classification",axis=1)

In [782]:
x

,id,age,bp,sg,al,su,rbc,pc,pcc,ba,...,hemo,pcv,wc,rc,htn,dm,cad,appet,pe,ane
0,383,80.0,80.000000,1.025,0.0,0.0,normal,normal,notpresent,notpresent,...,13.900000,49.000000,5100.000000,5.000000,no,no,no,good,no,no
1,0,48.0,80.000000,1.020,1.0,0.0,normal,normal,notpresent,notpresent,...,15.400000,44.000000,7800.000000,5.200000,yes,yes,no,good,no,no
2,291,47.0,80.000000,1.025,0.0,0.0,normal,normal,notpresent,notpresent,...,14.900000,41.000000,7000.000000,5.700000,no,no,no,good,no,no
3,316,35.0,76.469072,1.020,0.0,0.0,normal,normal,notpresent,notpresent,...,15.400000,48.000000,5000.000000,5.200000,no,no,no,good,no,no
4,342,44.0,60.000000,1.020,0.0,0.0,normal,normal,notpresent,notpresent,...,15.000000,50.000000,7700.000000,6.300000,no,no,no,good,no,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,123,43.0,80.000000,1.015,2.0,3.0,normal,abnormal,present,present,...,14.000000,42.000000,14900.000000,4.707435,no,no,no,good,no,no
396,91,56.0,70.000000,1.015,4.0,1.0,abnormal,normal,notpresent,notpresent,...,16.100000,52.000000,12500.000000,5.600000,no,no,no,good,no,no
397,64,55.0,80.000000,1.010,0.0,0.0,normal,normal,notpresent,notpresent,...,9.800000,38.884498,8406.122449,4.707435,no,no,no,good,no,no
398,173,17.0,70.000000,1.015,1.0,0.0,abnormal,normal,notpresent,notpresent,...,13.100000,41.000000,11200.000000,4.707435,no,no,no,good,no,no


###### 

In [783]:
y

0      notckd
1         ckd
2      notckd
3      notckd
4      notckd
        ...  
395       ckd
396       ckd
397       ckd
398       ckd
399    notckd
Name: classification, Length: 400, dtype: object

In [784]:
y=np.array([0 if value == 'notckd' else 1 for value in y])
y

array([0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1,
       1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0,
       1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0,
       1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0,
       0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1,
       1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0,
       0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1,
       1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0,
       0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0,
       1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1,

In [785]:
sum(y)/len(y)

0.625

In [786]:
unique_values_cat(x)

{'rbc': array(['normal', 'abnormal'], dtype=object),
 'pc': array(['normal', 'abnormal'], dtype=object),
 'pcc': array(['notpresent', 'present'], dtype=object),
 'ba': array(['notpresent', 'present'], dtype=object),
 'htn': array(['no', 'yes'], dtype=object),
 'dm': array(['no', 'yes'], dtype=object),
 'cad': array(['no', 'yes'], dtype=object),
 'appet': array(['good', 'poor'], dtype=object),
 'pe': array(['no', 'yes'], dtype=object),
 'ane': array(['no', 'yes'], dtype=object)}

# oneHot encoding

In [787]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

cols = [c for c in x.columns if x.dtypes[c]=='object']
columnTransformer = ColumnTransformer([('encoder', OneHotEncoder(drop='first'), cols)], remainder='passthrough')
data = columnTransformer.fit_transform(x)
column_name = columnTransformer.get_feature_names()
data =  pd.DataFrame(data, columns= column_name)
data.head(2)

,encoder__x0_normal,encoder__x1_normal,encoder__x2_present,encoder__x3_present,encoder__x4_yes,encoder__x5_yes,encoder__x6_yes,encoder__x7_poor,encoder__x8_yes,encoder__x9_yes,...,su,bgr,bu,sc,sod,pot,hemo,pcv,wc,rc
0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,119.0,46.0,0.7,141.000000,4.900000,13.9,49.0,5100.0,5.0
1,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,121.0,36.0,1.2,137.528754,4.627244,15.4,44.0,7800.0,5.2


# scaling the data

In [789]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
x = pd.DataFrame(scaler.fit_transform(data), columns=data.columns)

In [791]:
x

,encoder__x0_normal,encoder__x1_normal,encoder__x2_present,encoder__x3_present,encoder__x4_yes,encoder__x5_yes,encoder__x6_yes,encoder__x7_poor,encoder__x8_yes,encoder__x9_yes,...,su,bgr,bu,sc,sod,pot,hemo,pcv,wc,rc
0,0.364890,0.484322,-0.342518,-0.241249,-0.762252,-0.721743,-0.304789,-0.507801,-0.484322,-0.420084,...,-0.437797,-3.887651e-01,-2.321158e-01,-4.228624e-01,3.776065e-01,9.685073e-02,5.063316e-01,1.242555e+00,-1.311920e+00,3.485975e-01
1,0.364890,0.484322,-0.342518,-0.241249,1.311903,1.385535,-0.304789,-0.507801,-0.484322,-0.420084,...,-0.437797,-3.619875e-01,-4.352677e-01,-3.337432e-01,-9.275247e-15,1.261503e-15,1.059271e+00,6.283716e-01,-2.405187e-01,5.869017e-01
2,0.364890,0.484322,-0.342518,-0.241249,-0.762252,-0.721743,-0.304789,-0.507801,-0.484322,-0.420084,...,-0.437797,-3.218210e-01,-2.727461e-01,-3.693909e-01,2.688252e-01,9.685073e-02,8.749580e-01,2.598613e-01,-5.579709e-01,1.182662e+00
3,0.364890,0.484322,-0.342518,-0.241249,-0.762252,-0.721743,-0.304789,-0.507801,-0.484322,-0.420084,...,-0.437797,-6.565418e-01,-5.571589e-01,-4.585101e-01,-2.750810e-01,9.685073e-02,1.059271e+00,1.119719e+00,-1.351602e+00,5.869017e-01
4,0.364890,0.484322,-0.342518,-0.241249,-0.762252,-0.721743,-0.304789,-0.507801,-0.484322,-0.420084,...,-0.437797,-7.100972e-01,-2.321158e-01,-4.585101e-01,5.126273e-02,-1.517063e-01,9.118207e-01,1.365392e+00,-2.802002e-01,1.897575e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,0.364890,-2.064742,2.919556,4.145096,-0.762252,-0.721743,-0.304789,-0.507801,-0.484322,-0.420084,...,2.479925,-1.522134e-15,-5.571589e-01,-3.515671e-01,-9.275247e-15,1.261503e-15,5.431943e-01,3.826981e-01,2.576870e+00,-1.375768e-14
396,-2.740554,0.484322,-0.342518,-0.241249,-0.762252,-0.721743,-0.304789,-0.507801,-0.484322,-0.420084,...,0.534777,8.296188e-01,-6.384196e-01,-2.446240e-01,-1.662998e-01,-2.937388e-01,1.317310e+00,1.611066e+00,1.624513e+00,1.063510e+00
397,0.364890,0.484322,-0.342518,-0.241249,-0.762252,-0.721743,-0.304789,-0.507801,-0.484322,-0.420084,...,-0.437797,-2.726659e-02,-2.886963e-16,7.915374e-17,-9.275247e-15,1.261503e-15,-1.005037e+00,-5.236846e-15,4.330817e-15,-1.375768e-14
398,-2.740554,0.484322,-0.342518,-0.241249,-0.762252,-0.721743,-0.304789,-0.507801,-0.484322,-0.420084,...,-0.437797,-1.687482e+00,-1.136142e+00,7.535109e-01,8.127315e-01,-6.488202e-01,2.114305e-01,2.598613e-01,1.108653e+00,-1.375768e-14


# Train-Test Split

In [792]:
x_train, x_test , y_train , y_test = train_test_split(x,y, random_state =41,stratify=y)

In [793]:
model = KNeighborsClassifier()
model.fit(x_train,y_train)
y_pred =model.predict(x_test)


In [794]:
model.score(x_test, y_test)

0.97

In [ ]:
from sklearn.metrics import confusion_matrix
y_pred = model.predict(x_test)
cm = confusion_matrix(y_test, y_pred)
cm